### Flask
- 파이썬 코드를 사용하는 웹 프레임워크이다.
- `https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-i-hello-world`
- install
    - `pip install flask`

### 1. 프로젝트 생성
- hello

In [1]:
!mkdir -p hello
!mkdir -p hello/static
!mkdir -p hello/templates
!touch hello/hello.py
!touch hello/templates/index.html
!tree hello

# 이 코드가 실행이 되지않으면 직접 디렉토리랑 파일을 직접 생성해준다.

hello
├── hello.py
├── static
└── templates
    └── index.html

2 directories, 2 files


- hello.py : app 객체를 생성, route 설정을 하는 기능을 한다.
- static : js, css, image파일 등을 저장
- templates : html 코드를 저장하는 디렉토리
- 이렇게 크게 세가지로 구성되어있다.

- clf.pkl를 사용
- hello 폴더에 model 폴더를 생성후 clf.pkl를 업로드

In [2]:
%%writefile hello/hello.py
import pickle
import os # 경로를 찾을 때 사용
import numpy as np

from flask import *

app = Flask(__name__) # 객체를 생성

path = os.path.dirname(os.path.realpath(__file__)) # 파일이 위치한 디렉토리의 경로를 찾아준다.
with open("{}/model/clf.pkl".format(path), "rb") as f: # with : 파일을 open하고 자동으로 close를 해주는 문법
    model = pickle.load(f)

@app.route("/") # 루트
def hello():
    return "Hello Flask"

@app.route('/user/<name>')
def user(name):
    return render_template('index.html', name=name)

# api 생성기 작성
@app.route('/api/data')
def api_data():
    data = {"alice":25, "andy":35}
    return jsonify(data) # 딕셔너리 객체를 문자열로 바꾸어주는 함수

# api 추가
@app.route('/api/predic')
def predic():
    result = {"code": 200}
    
    # url을 get방식으로 데이터를 url에 넣어서 서버로 넣어준다.
    sentence = request.values.get("sentence")
    result["sentence"] = sentence
    result["category"] = ["정치", "경제", "사회", "생활/문화", "세계", "IT/과학"]
    
    # 예측되는 값을 넣어준다.
    # 확률 데이터가 리스트로 나온다. 결과데이터를 클라이언트에 보내줄것이다.
    # np.round_ : 파이썬에서 기본 제공되는 함수와의 중복을 피하기 위해서 사용
    result["result"] = list(np.round_(model.predict_proba([sentence])[0] * 100, 2))
    
    return jsonify(result) # 클라이언트로 보내준다.

app.run(debug=True)

Overwriting hello/hello.py


#### index.html에 html 코드를 작성
- jquery를 이용하여 버튼 설정

In [10]:
%%writefile hello/templates/index.html
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Hello Flask</title>
</head>
<body>
    Hello {{name}}
    <button class="result">Click!!</button>
    <div class="data"></div>
    <input type="text" class="sentence">
    <button type="button" class="prediction-btn">Predict!!!</button>
    <div class="prediction-wrap"></div>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.4.1/jquery.js"></script>
    <script type="text/javascript">
        $(document).ready(function(){
            $('.result').on('click', function(){
                $.getJSON("/api/data", function(data){
                    console.log(data);
                    $('.data').empty()
                    var tag = "<p>alice:" + data.alice + "</p>";
                        tag += "<p>andy:" + data.andy + "</p>";
                    $(".data").append(tag);
                })
            })
            $('.prediction-btn').on('click', function(){
                var sentence = $('.sentence').val();
                var url = "http://13.124.41.91/api/predic?sentence=" + sentence;
                $.getJSON(url, function(data){
                    console.log(data);
                    $('.prediction-wrap').empty()
                    for(var i = 0; i < data.category.length; i++){
                        var tag = "";
                            tag += "<p>" + data.category[i] + ":";
                            tag += data.result[i] + "%</p>";
                        $('.prediction-wrap').append(tag);
                    }
                })
            })
        })
    </script>
</body>
</html>

Overwriting hello/templates/index.html


### nginx 설정
- `sudo apt-get update`
- `sudo apt-get install nginx` - install
- `sudo systemctl status nginx` - 확인
- `aws console`에서 80번 포트 활성화
- 브라우저에 퍼블릭 IP 주소 입력해서 Welcome to nginx! 확인
- 설절파일 수정
    - `sudo vi /etc/nginx/sites-available/default`
    
```
# 1. 80을 8080으로 수정
server {
        listen 8080 default_server;
        listen [::]:8080 default_server;
        
# 2. 아래의 설정을 추가
server {
    listen 80;
    location / {
        proxy_pass http://localhost:5000;
    }
}
```

- nginx restart
    - `sudo systemctl restart nginx`